# Week 2: Exploratory Data Analysis (EDA) on the Kings County Housing Dataset

## Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.9 or later is installed (ideally I would generatlly recommend Python 3.10), as well as Scikit-Learn ≥ 1.0

In [5]:
# Python ≥3.9 is required
%pip install scikit-learn
%pip install statsmodels
import sys
assert sys.version_info >= (3, 9)

# Scikit-Learn ≥1.0 is required
import sklearn
assert sklearn.__version__ >= "1.0"

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Precision options
np.set_printoptions(precision=2)
pd.options.display.float_format = '{:.3f}'.format

# Statistical analysis and testing
from scipy import stats
from statsmodels.formula.api import ols
import statsmodels.api as sm

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 18.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 31.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Get the Data

First of all let's import the data from the CSV file.

In [6]:
housing = pd.read_csv(
    "/Users/harsimarbrar/Oxford/Courses/ml_ai_git_repo/mlai/kings_county_house_data.csv",
    dtype={'zipcode': str}   # US ZIP codes look like numbers but we want to treat them like strings
)

We can get an overall idea of the fields available using the `DataFrame.info()` and `DataFrame.describe()` methods.

In [7]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  object 
 17  lat            21613 non-null  float64
 18  long  

In [8]:
housing.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2395,8565900160,20150326T000000,995000.000,4,2.250,2340,13406,1.000,0,0,...,9,2340,0,1963,0,98040,47.538,-122.221,2340,10743
11189,1774220400,20140513T000000,591000.000,4,2.250,2710,38180,2.000,0,0,...,8,2710,0,1977,0,98077,47.770,-122.097,2590,38180
10978,8039900400,20140812T000000,375000.000,3,2.000,1670,13775,1.000,0,0,...,8,1670,0,1968,0,98045,47.487,-121.783,2130,14500
504,8906200070,20150210T000000,280000.000,3,1.500,1670,11610,1.000,0,0,...,7,1670,0,1963,0,98055,47.440,-122.191,1930,10200
11691,2426039247,20150325T000000,299950.000,2,1.500,1390,1756,3.000,0,0,...,7,1390,0,2005,0,98133,47.727,-122.357,1340,1756


In [9]:
housing.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15
count,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000,21613.000
mean,4580301520.865,540088.142,3.371,2.115,2079.900,15106.968,1.494,0.008,0.234,3.409,7.657,1788.391,291.509,1971.005,84.402,47.560,-122.214,1986.552,12768.456
std,2876565571.312,367127.196,0.930,0.770,918.441,41420.512,0.540,0.087,0.766,0.651,1.175,828.091,442.575,29.373,401.679,0.139,0.141,685.391,27304.180
min,1000102.000,75000.000,0.000,0.000,290.000,520.000,1.000,0.000,0.000,1.000,1.000,290.000,0.000,1900.000,0.000,47.156,-122.519,399.000,651.000
25%,2123049194.000,321950.000,3.000,1.750,1427.000,5040.000,1.000,0.000,0.000,3.000,7.000,1190.000,0.000,1951.000,0.000,47.471,-122.328,1490.000,5100.000
50%,3904930410.000,450000.000,3.000,2.250,1910.000,7618.000,1.500,0.000,0.000,3.000,7.000,1560.000,0.000,1975.000,0.000,47.572,-122.230,1840.000,7620.000
75%,7308900445.000,645000.000,4.000,2.500,2550.000,10688.000,2.000,0.000,0.000,4.000,8.000,2210.000,560.000,1997.000,0.000,47.678,-122.125,2360.000,10083.000
max,9900000190.000,7700000.000,33.000,8.000,13540.000,1651359.000,3.500,1.000,4.000,5.000,13.000,9410.000,4820.000,2015.000,2015.000,47.778,-121.315,6210.000,871200.000


##### Description of the features:

Here follows a detailed description of all the features (i.e. columns/variables) in the dataset.

* **id** - unique identifier for a house
* **date** - house was sold
* **price** - price, our prediction target
* **bedrooms** - number of Bedrooms/House
* **bathrooms** - number of bedrooms
* **sqft_living** - square footage of the home
* **sqft_lot** - square footage of the entire lot
* **floors** - total number of floors (levels) in house
* **waterfront** - house which has a view to a waterfront
* **view** - quality of view
* **condition** - how good the condition is ( overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbours
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbours

## Question
**Question:** which of these variables are quantitaive/numerical, ordinal, and categorical?



## Business problem

We want to accurately predict the housing prices in the Kings county (Washington, US) based on the iformation available in the dataset.

## 2. Create a Training and Test Set

Splitting the original dataset into training and test set should be done even before you start a more thorough Exploratory Data Analysis (EDA).

Creating a test set is theoretically simple: pick some instances randomly, typically 20% of the dataset (or less if your dataset is very large), and set them aside. We will use a function from `scikit-learn` which splits a dataset into training and test set.

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
train_set.shape, test_set.shape

### 2.1 Stratified split

Using `train_test_split()` we would just be doing a simple randomized sampling. But this might not be a representative sampling of the whole dataset, if we do not preserve the proportions (or percentages) of significant input features. Let's hypothesize that we learned from expert the `sqft_living` field is an important predictor for the house price. 

In [ ]:
housing.sqft_living.hist(bins=100, figsize=(14,10))
plt.show()

A way to preserve the same proportion of samples with respect to `sqft_living` is to use a stratified split. We can split the dataset in such a way that the proportion of samples wrt `sqft_living` is preserved across the training and test set. To do this we first need to convert `sqft_living` into a categorical/ordinal variable.

In [ ]:
housing["sqft_living_cat"] = pd.cut(
    housing.sqft_living, 
    bins=[0., 1000., 2000., 3000., 4000., np.inf],
    labels=[1, 2, 3, 4, 5]
)
housing['sqft_living_cat'].hist()
plt.show()

In [ ]:
housing['sqft_living_cat'].value_counts() / len(housing)

Scikit-learn offers the class `StratifiedShuffleSplit` to perform stratified splits.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in splitter.split(housing, housing.sqft_living_cat):
    train_set = housing.loc[train_index]
    test_set = housing.loc[test_index]

In [ ]:
train_set.shape, test_set.shape

We can verify that the proportion of samples wrt `sqft_living_cat` is preserved in training and test set:

In [ ]:
train_set.sqft_living_cat.value_counts() / len(train_set)

In [ ]:
test_set.sqft_living_cat.value_counts() / len(test_set)

We can now remove the `sqft_living_cat` feature as we will not need it again.

In [ ]:
for set_ in (train_set, test_set):
    set_.drop("sqft_living_cat", axis=1, inplace=True)

## 3. Discover and Visualize the Data to Gain Insights

### 3.1 Outlier Detection

In [ ]:
p = train_set[
    ['price', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 
     'lat', 'long', 'sqft_living15', 'sqft_lot15']
].plot.box(subplots=True, layout=(3, 3), figsize=(18,18))

In [ ]:
_ = train_set[['price']].boxplot(figsize=(15,10))

Box plot use the IQR method to display data and outliers(shape of the data) but in order to be get a list of identified outlier, we will need to use the mathematical formula and retrieve the outlier data.

Wikipedia Definition:
_The interquartile range (IQR), also called the midspread or middle 50%, or technically H-spread, is a measure of statistical dispersion, being equal to the difference between 75th and 25th percentiles, or between upper and lower quartiles, $IQR = Q_3 − Q_1$.

In other words, the IQR is the first quartile subtracted from the third quartile; these quartiles can be clearly seen on a box plot on the data.

It is a measure of the dispersion similar to standard deviation or variance, but is much more robust against outliers._

If a data point is below $Q_1 - 1.5\times IQR$ or above $Q_3 + 1.5\times IQR$ then it's an outlier.

![box-plot-summary](../images/box-plot-iqr.png)




## Exercise 1 
<b>Exercise 1:</b> Compute for me the count of outliers in our training set with respect to the `price` feature. (Hint: check the `DataFrame.quantile()` method and find a way to count the occurrences of values in a column of a DataFrame.) Additionally, write the code to remove those outliers. 

In [ ]:
## Write you solution here. Add as many cells as you see fit


Are the outliers legitimate or should we remove them?

### 3.2 Visualize geographical data

In [ ]:
train_set.plot(
    kind="scatter", x="long", y="lat", figsize=(15,10)
)
plt.show()

In [ ]:
train_set.plot(kind="scatter", x="long", y="lat", alpha=0.1, figsize=(15,10))
plt.show()

We can add some info to the plot:
    * scale the size of the markers based on the surface areas of the house
    * colour-code the dots based on the house price

In [ ]:
train_set.plot(
    kind="scatter",
    x="long",
    y="lat",
    alpha=0.1,
    figsize=(20,13),
    s=train_set["sqft_living"]/100,
    label="sqft_living",
    c="price",
    cmap=plt.get_cmap('jet'),
    colorbar=True
)
plt.legend()
plt.show()

Using the same colormap (i.e. jet), we can try to improve the visualization above, setting an upper value that is reasonable, (i.e less or equal to QR3 + 1.5 IQR such as 1,000,000 $), and not the highest value in the range.

We can create a custom discrete colorbar by using `matplotlib.colors.BoundaryNorm` as normalizer for your scatterplot. See the norm argument in `matplotlib.pyplot.scatter()`: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html


In [ ]:
cmap = plt.cm.jet  # define the colormap
bounds = np.linspace(0, q3.price + 1.5 * iqr.price, 11) # define 11 evenly space

# The matplotlib.colors.BoundaryNorm class is used to create a colormap based on discrete numeric intervals.
norm = mpl.colors.BoundaryNorm(
    bounds, # Monotonically increasing sequence of boundaries
    cmap.N # Number of colors in the colormap to be used
)

plt.figure(figsize=(20, 13))
plt.scatter(
    x=train_set["long"],
    y=train_set["lat"],
    alpha=0.1,
    s=train_set["sqft_living"]/100, # size of the dot
    label=train_set["sqft_living"],
    c=train_set["price"], # colour of the dot
    cmap=cmap, # colour map 
    norm=norm # used to scale the color data, c, in the range 0 to 1, in order to map into the colormap cmap
)
plt.colorbar(label="Price", orientation="vertical")
plt.show()

An alternative way to achieve the same result as above is to use a `matplotlib.colors.LinearSegmentedColormap`:

In [ ]:
cmap = plt.cm.jet  # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N
)

# define the bins and normalize
bounds = np.linspace(0, q3.price + 1.5 * iqr.price, 11)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
plt.figure(figsize=(20, 13))
plt.scatter(
    x=train_set["long"], y=train_set["lat"],
    alpha=0.1,
    s=train_set["sqft_living"]/100, label=train_set["sqft_living"],
    c=train_set["price"], cmap=cmap, norm=norm
)
plt.colorbar(label="Price", orientation="vertical")
plt.show()

## Question
**Question:** What insights can we infer from this graph?



## Exercise 2
<b>Exercise 2:</b> explore on your own other ways to improve the graph above. You could look for ways to overlap it on top of the county map, or you could see if you can encode information differently.

Map support can be provided using either [Basemap](https://basemaptutorial.readthedocs.io/en/latest/) or [folium](https://python-visualization.github.io/folium/).
* To install basemap: `conda install -c conda-forge basemap basemap-data-hires`
* To install folium: `conda install -c conda-forge folium`

In [ ]:
### Write here your possible solution



### 3.2 Numerical features: looking for correlations

The dataset is not that big, and we can compute the standard correlation coefficient (Pearson’s r coefficient) between every two features using the `DataFrame.corr()` method:

In [ ]:
corr_matrix = train_set.corr(numeric_only=True)
corr_matrix["price"].sort_values(ascending=False)

N.B. The correlation coefficient only measures linear correlations, and it may completely miss nonlinear correlation factors. 

Another way to check for correlation visually is to use the `scatter_matrix()` utility function offered by Pandas, which leverages `matplotlib`, or seaborn's `pairplot()` function.

In [ ]:
attributes = [
    "price", "sqft_living", "grade",
    "sqft_above", "sqft_living15", "bathrooms"
]
pd.plotting.scatter_matrix(
    train_set[attributes], figsize=(15, 10)
)
plt.show()

In [ ]:
ax = sns.pairplot(train_set[attributes])

### 3.2 Numerical features: checking for multicolinearity

In [ ]:
# generate copy of df without target variable (price), id, date and lat/lon to perform multicolinearity check
train_set_pred = train_set.drop(["id", "date", "lat", "long"], axis=1)

# check for multicolinearity with seaborn heatmap
# compute the correlation matrix
corr = round(train_set_pred.corr(numeric_only=True), 3)

# set up the matplotlib figure
fig, ax = plt.subplots(figsize=(15, 12))

# generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
ax = sns.heatmap(corr, cmap=cmap, vmax=1, vmin=-1, center=0, square=True, linewidths=1, cbar_kws={"shrink": .75}, annot=True)

plt.show()

As the correlation matrix is simmetrical we can get rid of the upper triangle using a triangular boolean mask: 

In [ ]:
# generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Let's visualise the mask with matplotlib
fig, ax = plt.subplots()

ax.matshow(mask, cmap="gray")

for (i, j), z in np.ndenumerate(mask):
    ax.text(j, i, '{:0.0f}'.format(z), ha='center', va='center', c="gray")

plt.show()

In [ ]:
# set up the matplotlib figure
fig, ax = plt.subplots(figsize=(15, 12))

# Draw the heatmap with the mask and correct aspect ratio
ax = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, vmin=-1, center=0, square=True, linewidths=1, cbar_kws={"shrink": .75}, annot=True)

plt.show()

## Question 
**Question:** what conclusions can we draw from this correlation matrix? Is there any redundant information in the dataset?


### 3.3 Preparing numerical data for training:

In [ ]:
numerical_features = ['sqft_living','sqft_lot','sqft_above','sqft_basement','yr_built','sqft_living15','sqft_lot15']

for feat in numerical_features:
    sns.jointplot(
        x=feat,
        y="price",
        data=train_set,
        kind='reg',
        label=feat,
        joint_kws={'line_kws':{'color':'orange'}}
    )
    plt.legend()
    plt.show()

As we have already seen in the multicolinearity test `sqft_living`, `sqft_living15`, and `sqft_above` predictors all seem to have prominent linear correlation with price, Of the three, we will use `sqft_living` as it is the one that mostly correlates with price. 

We can notice that there are a bunch of zeros in `sqft_basement`. That indicates that those homes do not have basements.To tackle that, we will create a binary feature that indicates whethere a house has a basement or not.

Finally, `yr_built` does not seem to show any clear relationship with price. We could combine the information of `yr_built` with `yr_renovated` to see get a more useful information. We will create a `renovated` binary flag. If a house is older than 25 years (relative to the most recent data in the dataset) and has not been renovated we will set `renovated` to 0, otherwise to 1.

### 3.4 Categorical and Ordinal Features

Let's examine the categorical and ordinal features more closely. Boxplots and violin plots are the most suited type of graphs for plotting a categorical variable against a numerical one (i.e. our target variable `price`)

First we need to convert these columns to the category type:

In [ ]:
categorical_features = ['bedrooms','bathrooms','floors','waterfront','view','condition','grade','zipcode']
train_set[categorical_features] = train_set[categorical_features].astype('category')
train_set.info()

We can then start using boxplots or violinplots to further investigate targeted correlations, such as 'grade' vs 'price' or 'floors' vs 'price'.

## Exercise 3

<b>Exercise 3:</b> write a function that takes a categorical or ordinal feature as a first argument, the size of a figure as a second argument and plots, using seaborn, a set of boxplots of the price distribution for each category in the input categorical feature.

In [ ]:
def print_boxplot(
    data_set: pd.DataFrame,
    feature: str, 
    figsize: tuple[int, int] = (14, 6), 
    xlabels_rotation: int = None
):
    """
    data_set: the dataset with the feature and price columns
    feature: the name of the feature to plot against "price"
    figsize: the (width, height) size of the figure
    xlabels_rotation: the rotation of the X labels. Default is equivalent to no rotation
    """
    # create a dataframe containing only feature and "price"
    data = ...
    # specify the size of the figure
    ...
    # create the feature vs price boxplot using seaborn. Check the seaborn boxplot documentation
    chart = ...
    # if an x-label rotation is provided, rotate the x label of the chart
    if xlabels_rotation is not None:
        chart.set_xticklabels(chart.get_xticklabels(), rotation=xlabels_rotation, horizontalalignment='center')

In [ ]:
## Try your function to plot 'grade' vs 'price'.
## If you have implemented it correctly it will plot out the boxplots
print_boxplot(train_set, 'grade')

In [ ]:
## Try your function to plot 'floors' vs 'price'.
## If you have implemented it correctly it will plot out the boxplots
print_boxplot(train_set, 'floors', figsize=(12, 12))

In [ ]:
## Try your function to plot 'bathrooms' vs 'price'.
## If you have implemented it correctly it will plot out the boxplots
print_boxplot(train_set, 'bathrooms', figsize=(20, 12))

## Question 
**Question:** Do you notice anything suspicious with bathrooms?


In [ ]:
print_boxplot(train_set, 'bedrooms', figsize=(20, 12))

## Question 
**Question:** Do you notice anything suspicious with bedrooms?


In [ ]:
print_boxplot(train_set, "view")

In [ ]:
print_boxplot(train_set, "waterfront")

In [ ]:
print_boxplot(train_set, "zipcode", figsize=(24, 8), xlabels_rotation= 60)

### 3.4 Categorical variables: statistical test to evaluate significance (optional)

#### ANOVA test for the "view" feature

Analysis of Variance (ANOVA) is a statistical test that analyses the difference among means. ANOVA checks if the means of two or more groups are significantly different from each other. In our case we want to see if the average prices are significantly different among the 5 view categories.

We can compute ANOVA using the `statsmodels` library.

In [ ]:
lin_mod = ols("price ~ view", train_set).fit()
table = sm.stats.anova_lm(lin_mod, typ=2)
print(table)

As the p-value for view is 0.000 < 0.05, the difference of average house prices among the view categories is significant.

#### Post-hoc test

We can run a post-hoc test to make pairwise comparisons and see which pairs of view categories have significant mean differences:

In [ ]:
pair_t = lin_mod.t_test_pairwise("view")
pair_t.result_frame

All the differences are significant except between view 1 and view 2. This could already be discerned from the boxplot graph. We could then rework the view categories merging together views 1 and 2 and reducing the 5 categories to 4.

Another option would be convert this feature to a binary one ("view" vs "no view"). For our model training we will keep all the view levels and treat it as an ordinal variable.

### ANOVA and post-hoc tests on ZIPCODE

We could do a symilar analysis on the `zipcode` field to reduce the number of categories there.

In [ ]:
zipcodes = train_set["zipcode"].unique()
zipcodes

We have 70 zipcodes. That is a bit too many categories given the number of samples we have. A plurality of these zipcodes likely have very few samples so we may encur into overfitting our models. We could find a way to cluster/collapse these zipcodes together, based on whether
their average house price is significantly different or not.
To do this we can first do an ANOVA test on `zipcode` and their peform pairwise post-hoc t-test and then collapse together those zipcodes whose t-test indicates that the means are not significantly different.

In [ ]:
lin_mod = ols("price ~ zipcode", train_set).fit()
table = sm.stats.anova_lm(lin_mod, typ=2)
print(table)

As the p-value for view is 0.000 < 0.05, the difference of average house prices among the view categories is significant.

#### Post-hoc test

In [ ]:
pair_t = lin_mod.t_test_pairwise("zipcode")
pair_t.result_frame

Let's now extract the zipcode pairs whose post-hoc t-test fails to reject the null hypothesis (i.e. the average prices are not significantly different)

In [ ]:
ttests_frame = pair_t.result_frame[pair_t.result_frame["reject-hs"].isin([False])].reset_index(names="zipcode-pairs")

In [ ]:
ttests_frame["zipcode-pairs"] = ttests_frame["zipcode-pairs"].str.split("-")

In [ ]:
ttests_frame

Now we'll group together all the zipcodes for which the mean house price is not significantly different. In doing so, we will create a new category named `zipcode_group`

In [ ]:
def make_zipcode_groups(data_set: pd.DataFrame) -> list[set]:
    """
    Groups together zipcodes with similar house prices
    :param data_set: it can be the training or test set
    :returns: he generated zipcode groups. Each zipcode group is a set
    """
    # firsr we perform a post-hoc t-test with correction for each zipcode pair
    pair_t = ols("price ~ zipcode", train_set).fit().t_test_pairwise("zipcode")
    # we retain all the zipcode pairs for which the post-hoc test fails to reject the null hypothesis
    # these are all the zipcode pairs whose average house price is not significantly different with a 95% CI
    t_tests_frame = pair_t.result_frame[pair_t.result_frame["reject-hs"].isin([False])].reset_index(names="zipcode-pairs")
    # we extract the zipcode pairs as a list
    zipcode_pairs = t_tests_frame["zipcode-pairs"].str.split("-").tolist()
    # we group together zipcodes with "similar" average house prices
    zipcode_groups = []
    for zipcode in train_set["zipcode"].unique():
        if any(zipcode in group for group in zipcode_groups):
            continue
        new_group = { zipcode }
        for pair in zipcode_pairs:
            if zipcode in pair:
                new_group.update(pair)
        zipcode_groups.append(new_group)
    return zipcode_groups 

## Exercise 4
**Exercise 4:** Let's test the function `make_zipcode_groups()` on the training set.

In [ ]:
# write your solution here


We have reduced our 70 zipcodes to just 9 zipcode groups. This should not impact too much our predictive power wrt `price` while reducing the risk of overfitting.

We can now assign each sample in the training and test set a `zipcode_group` feature based on its zipcode:

In [ ]:
def assign_to_zipcode_group(zipcode: str, zipcode_groups: list[set], group_prefix="zg") -> str:
    """
    given a zipcode and a list of groups it assign a zipcode to a group
    The groups will be called using the group_prefix as a prefix
    """
    try:
        res = next(i for i, group in enumerate(zipcode_groups) if zipcode in group)
        return f"{group_prefix}_{res}"
    except StopIteration:
        # we need to take into account that some zipcode may be missing in the training set and present in the test set
        return f"{group_prefix}_other"

In [ ]:
train_set["zipcode_group"] = train_set["zipcode"].apply(assign_to_zipcode_group, zipcode_groups=zipcode_groups)

In [ ]:
train_set

In [ ]:
train_set["zipcode_group"].unique()

In [ ]:
train_set["zipcode_group"].value_counts()

Note: as an expected consequence of our clustering, some zipcode groups are now over-represented in the training set

In [ ]:
test_set["zipcode_group"] = test_set["zipcode"].apply(assign_to_zipcode_group, zipcode_groups=zipcode_groups)

In [ ]:
train_set["zipcode_group"].unique()

As it turns out all the zipcodes were present in the training set. 

### 4. Conclusions of the EDA:

At the end of our EDA we have reached a few conclusions:

* We have identified quantitative, ordinal and categorical variables.
* We have identified a cutoff at latitude  ~47.5 between more expensive houses and cheaper houses.We will create a binary engineered feature to capture this. We could also create a cutoff at -126.1 long to separate the urban west from the rural east of the county. We will then remove `lat` and `long`.
* We have decided to discard `sqft_living15` and `sqft_living_above` in favour of `sqft_living`
* We have decided to add a binary engineered feature that indicates whether a house has a basement or not. We will then remove the continuous variable `sqft_basement`
* We will create a renovated binary flag. If a house is older than 25 years (relative to the most recent data in the dataset) and has not been renovated we will set renovated to 0, otherwise to 1. We will then remove the continuous variable `yr_built` and `yr__renovated`
* We have decided to collapse the 70 zipcodes into 9 zipcode groups based on average house prices in the zipcodes 
* Some houses report 0 bathrooms. We need to replace those values with more meaningful estimates.
* One house has 33 bedrooms. We will replace that value with 3, as it looks like a reporting mistake.